# Object Detection in colab : Training

##1 . Check out : GPU

In [ ]:
!nvidia-smi

##2. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##3. Unzip picture_data
(for train&test)

In [ ]:
%cd /content/drive/MyDrive
!mkdir yolo_custom_model_Training

In [ ]:
!ls '/content/drive/MyDrive/yolo_custom_model_Training'

In [ ]:
!unzip '/content/drive/MyDrive/yolo_custom_model_Training/custom_data.zip' -d '/content/drive/MyDrive/yolo_custom_model_Training'

##4. Get AlexeyAB/darknet
(github opensource)


In [ ]:
%pwd

In [ ]:
# download dataset in current directory(above path)
!git clone 'https://github.com/AlexeyAB/darknet.git' '/content/drive/MyDrive/yolo_custom_model_Training/darknet'

In [ ]:
# move current directory to darknet
%cd /content/drive/MyDrive/yolo_custom_model_Training/darknet

In [ ]:
# change setting values in 'Makefile' file 
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
# Compile model
"""  take care do not disconnect : file directory may be interupted 
if your network down during compile, I recommend delete darknet folder and restart number 4(get AlexeyAB/darknet)"""

!make

In [ ]:
%cd ..
!darknet/darknet

##5. Create Train&Test dataset

In [ ]:
!git clone 'https://github.com/jakkcoder/training_yolo_custom_object_detection_files' '/content/drive/MyDrive/yolo_custom_model_Training/training_yolo_custom_object_detection_files-main'

In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training/training_yolo_custom_object_detection_files-main

In [ ]:
# check out current dir files
!ls

In [ ]:
# copy creating-train-and-test-txt-files.py & creating-files-data-and-name.py
"""creating-train-and-test-txt-files.py >> create 'train.txt' & 'test.txt' files
   creating-files-data-and-name.py >> create label 'labelled_data.data' file
   <if you excute both .py files, you get mentioned files upper lines 2,3>"""

!cp creating-train-and-test-txt-files.py /content/drive/MyDrive/yolo_custom_model_Training/custom_data
!cp creating-files-data-and-name.py /content/drive/MyDrive/yolo_custom_model_Training/custom_data

In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training/custom_data

In [ ]:
# change paths in both .py files
!sed -i '39 s@/home/my_name/Downloads/video-to-annotate@custom_data@' creating-train-and-test-txt-files.py
!sed -i '74 s@jpeg@jpg@' creating-train-and-test-txt-files.py
!sed -i '36 s@/home/my_name/Downloads/video-to-annotate@custom_data@' creating-files-data-and-name.py

In [ ]:
# move current dir one step before
%cd ..

In [ ]:
# excute .py file >> 'train.txt', 'test.txt'   
!python custom_data/creating-train-and-test-txt-files.py

In [ ]:
# excute .py file >> 'labelled_data.data'
!python custom_data/creating-files-data-and-name.py

##6. Setting diretory & values
(yolov4.conv.137 & yolov4_custom.cfg)

###6.1 pre-trained model

In [ ]:
# create directory 'custom_weight'
!mkdir custom_weight

In [ ]:
# move 'yolov4.conv.137' file to 'custom_weight' dir
!mv yolov4.conv.137 custom_weight/

###6.2 change values

In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training/darknet/cfg

In [ ]:
# copy yolov4.cfg file & rename & paste
!cp yolov4.cfg yolov4_custom.cfg

In [ ]:
# change values for training
!sed -i '2 s@batch=64@batch=8@' yolov4_custom.cfg

!sed -i '7 s@width=608@width=416@' yolov4_custom.cfg
!sed -i '8 s@height=608@height=416@' yolov4_custom.cfg  

!sed -i '19 s@500500@10000@' yolov4_custom.cfg  #maxbatch=class*2000
!sed -i '21 s@400000,450000@8000,9000@' yolov4_custom.cfg  #maxbatch*0.8, maxbatch*0.9

!sed -i '968 s@classes=80@classes=5@' yolov4_custom.cfg
!sed -i '1056 s@classes=80@classes=5@' yolov4_custom.cfg
!sed -i '1144 s@classes=80@classes=5@' yolov4_custom.cfg

!sed -i '961 s@filters=255@filters=30@' yolov4_custom.cfg  #filters=(4+1+classes)*3 
!sed -i '1049 s@filters=255@filters=30@' yolov4_custom.cfg
!sed -i '1137 s@filters=255@filters=30@' yolov4_custom.cfg

###6.3 create back dir

In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training

In [ ]:
!mkdir backup

##7. Train the model 


In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training

In [ ]:
!darknet/darknet detector train custom_data/labelled_data.data darknet/cfg/yolov4_custom.cfg custom_weight/yolov4.conv.137 -dont_show

#Webcam object detection

##8. Check performance

In [ ]:
# define helper function imShow
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  #plt.show('')


In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training
!ls

In [ ]:
# only works if the training does not get interrupted 
imShow('chart.png')

In [ ]:
%cd custom_data

!sed -i 's@custom_data@/content/drive/MyDrive/yolo_custom_model_Training/custom_data@' test.txt
!sed -i 's@custom_data@/content/drive/MyDrive/yolo_custom_model_Training/custom_data@' train.txt

!sed -i 's@custom_data@/content/drive/MyDrive/yolo_custom_model_Training/custom_data@' labelled_data.data
!sed -i '5 s@.*@backup = /content/drive/MyDrive/yolo_custom_model_Training/backup/@' labelled_data.data

!cat labelled_data.data

In [ ]:
%cd /content/drive/My Drive/yolo_custom_model_Training/darknet
!chmod +x ./darknet

##8.1 Check mAP

In [ ]:
#You can check the mAP for all the saved weights to see which gives the best results

!./darknet detector map /content/drive/MyDrive/yolo_custom_model_Training/custom_data/labelled_data.data /content/drive/MyDrive/yolo_custom_model_Training/darknet/cfg/yolov4_custom.cfg /content/drive/MyDrive/yolo_custom_model_Training/backup/yolov4_custom_10000.weights -points 0

##9. Test custom Object Detection

In [ ]:
#set your custom cfg to test mode 
%cd /content/drive/MyDrive/yolo_custom_model_Training/darknet/cfg
!sed -i 's/batch=64/batch=1/' yolov4_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4_custom.cfg
%cd ..

##9.1 Run detector on an image

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, the thresh flag sets the minimum accuracy required for object detection)

!./darknet detector test /content/drive/MyDrive/yolo_custom_model_Training/custom_data/labelled_data.data /content/drive/MyDrive/yolo_custom_model_Training/darknet/cfg/yolov4_custom.cfg /content/drive/MyDrive/yolo_custom_model_Training/backup/yolov4_custom_10000.weights /content/drive/MyDrive/yolo_custom_model_Training/custom_data/1.jpg -thresh 0.3
imShow('predictions.jpg')

## 9.2 Run detector on a webcam image

In [ ]:
#Run detector on images captured by webcam for your custom YOLOv4-tiny trained model
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

!./darknet detector test /content/drive/MyDrive/yolo_custom_model_Training/custom_data/labelled_data.data /content/drive/MyDrive/yolo_custom_model_Training/darknet/cfg/yolov4_custom.cfg /content/drive/MyDrive/yolo_custom_model_Training/backup/yolov4_custom_10000.weights photo.jpg -thresh 0.5
imShow('predictions.jpg')   

## 9.3 Run detector on a video

In [ ]:
# run your custom detector on a video with this command (upload a video to your google drive to test, the thresh flag sets the minimum accuracy required for object detection).This saves the output video with the detections in your output path

#!./darknet detector demo data/obj.data cfg/yolov4-tiny-custom.cfg /mydrive/yolov4-tiny/training/yolov4-tiny-custom_best.weights -dont_show /mydrive/mask_test_videos/test3.mp4 -i 0 -out_filename /mydrive/mask_test_videos/result.avi 

## 9.4 Run detector on a live webcam

In [ ]:
# Code from theAIGuysCode Github (https://github.com/theAIGuysCode/YOLOv4-Cloud-Tutorial/blob/master/yolov4_webcam.ipynb)
#adjusted for my custom YOLOv4-tiny trained weights, config and obj.data files

# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline


# import darknet functions to perform object detections
from darknet import *
# load in our YOLOv4 architecture network
network, class_names, class_colors = load_network("/content/drive/MyDrive/yolo_custom_model_Training/darknet/cfg/yolov4_custom.cfg", 
                                                  "/content/drive/MyDrive/yolo_custom_model_Training/custom_data/labelled_data.data", 
                                                  "/content/drive/MyDrive/yolo_custom_model_Training/backup/yolov4_custom_10000.weights")
width = network_width(network)
height = network_height(network)

# darknet helper function to run detection on image
def darknet_helper(img, width, height):
  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height),
                              interpolation=cv2.INTER_LINEAR)

  # get image ratios to convert bounding boxes to proper size
  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  # run model on darknet style image to get detections
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  return detections, width_ratio, height_ratio

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes  

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # call our darknet helper on video frame
    detections, width_ratio, height_ratio = darknet_helper(frame, width, height)

    # loop through detections and draw them on transparent overlay image
    for label, confidence, bbox in detections:
      left, top, right, bottom = bbox2points(bbox)
      left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
      bbox_array = cv2.rectangle(bbox_array, (left, top), (right, bottom), class_colors[label], 2)
      bbox_array = cv2.putText(bbox_array, "{} [{:.2f}]".format(label, float(confidence)),
                        (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        class_colors[label], 2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes   